In [627]:
import numpy as np
import pandas as pd

In [ ]:
train = pd.read_csv('Restaurant.csv')
print (train.info())

In [ ]:
print(train.head(10))

In [ ]:
nptest = np.array(train["WillWait"])
# nptest.sort()
nptest

In [634]:
YesNo_Mapping = {
    "Yes": 1,
    "No": 0,
}

train["Alt"] = train["Alt"].replace(YesNo_Mapping)
train["Bar"] = train["Bar"].replace(YesNo_Mapping)
train["Fri"] = train["Fri"].replace(YesNo_Mapping)
train["Hun"] = train["Hun"].replace(YesNo_Mapping)
train["Rain"] = train["Rain"].replace(YesNo_Mapping)
train["Res"] = train["Res"].replace(YesNo_Mapping)
train["WillWait"] = train["WillWait"].replace(YesNo_Mapping)

Pat_Mapping = {
    "None": 0,
    "Some": 0.5,
    "Full": 1,
}

train["Pat"] = train["Pat"].replace(Pat_Mapping)

Price_Mapping = {
    "$": 1,
    "$$": 2,
    "$$$": 3,
}
train["Price"] = train["Price"].replace(Price_Mapping)

Type_Mapping = {
    "French": 1,
    "Thai": 2,
    "Burger": 3,
    "Italian": 4,
}
train["Type"] = train["Type"].replace(Type_Mapping)

Est_Mapping = {
    "0-10": 1,
    " 10-30": 2,
    "30-60": 3,
    ">60": 4,
}
train["Est"] = train["Est"].replace(Est_Mapping)

In [635]:
# print(train[["Type", "WillWait"]].groupby(["Type"], as_index=False).mean())
GroupedByType = train[["Type", "WillWait"]].groupby(["Type"], as_index=False).mean()

print(GroupedByType)

   Type  WillWait
0     1       0.5
1     2       0.5
2     3       0.5
3     4       0.5


In [636]:
GroupedByPat = train[["Pat", "WillWait"]].groupby(["Pat"], as_index=False).mean()

print(GroupedByPat)

   Pat  WillWait
0  0.5  1.000000
1  1.0  0.333333


In [637]:
print (train)

    id  Alt  Bar  Fri  Hun  Pat  Price  Rain  Res  Type  Est  WillWait
0    1    1    0    0    1  0.5      3     0    1     1    1         1
1    2    1    0    0    1  1.0      1     0    0     2    3         0
2    3    0    1    0    0  0.5      1     0    0     3    1         1
3    4    1    0    1    1  1.0      1     1    0     2    2         1
4    5    1    0    1    0  1.0      3     0    1     1    4         0
5    6    0    1    0    1  0.5      2     1    1     4    1         1
6    7    0    1    0    0  NaN      1     1    0     3    1         0
7    8    0    0    0    1  0.5      2     1    1     2    1         1
8    9    0    1    1    0  1.0      1     1    0     3    4         0
9   10    1    1    1    1  1.0      3     0    1     4    2         0
10  11    0    0    0    0  NaN      1     0    0     2    1         0
11  12    1    1    1    1  1.0      1     0    0     3    3         1


In [638]:
class Node:
    def __init__(self, attribute=None, value=None, label=None):
        self.attribute = attribute  # Attribute at this node
        self.value = value  # Value of the attribute (for non-leaf nodes)
        self.label = label  # Class label (for leaf nodes)
        self.children = {}  # Dictionary to store child nodes

    def add_child(self, value, child_node):
        self.children[value] = child_node

In [639]:
def PLURALITY_VALUE(examples):
    return np.argmax(np.bincount(examples[:, -1]))

In [640]:
def find_attribute_index(attribute, attributes):
    attribute_list = list(attributes)
    index = attribute_list.index(attribute)
    return index

In [641]:
def select_best_attribute_entropy(attributes, examples):
    best_attribute = None
    min_entropy = np.inf

    for attribute in attributes:
        entropy = entropy_help(examples[:, find_attribute_index(attribute, attributes)])
        if entropy < min_entropy:
            best_attribute = attribute
            min_entropy = entropy

    return best_attribute


def entropy_help(examples):
    unique_values, counts = np.unique(examples, return_counts=True)
    probabilities = counts / len(examples)
    entropy = -np.sum(probabilities * np.log2(probabilities))
    return entropy

In [643]:
def select_best_attribute_GiniIndex(attributes, examples):
    best_attribute = None
    best_gini_index = float('inf')

    for attribute in attributes:
        attribute_values = np.unique(examples[:, find_attribute_index(attribute, attributes)])
        attribute_gini_index = 0

        for value in attribute_values:
            value_examples = examples[examples[:, find_attribute_index(attribute, attributes)] == value]
            value_prob = len(value_examples) / len(examples)
            value_gini_index = GiniIndex_help(value_examples[:, -1])

            attribute_gini_index += value_prob * value_gini_index

        if attribute_gini_index < best_gini_index:
            best_gini_index = attribute_gini_index
            best_attribute = attribute

    return best_attribute

def GiniIndex_help(labels):
    label_counts = np.bincount(labels)
    label_probs = label_counts / len(labels)
    gini_index = 1 - np.sum(label_probs ** 2)
    return gini_index

In [650]:
def LEARN_DECISION_TREE(examples, attributes, parent_examples):
    if len(examples) == 0:
        return Node(label=PLURALITY_VALUE(parent_examples))
    
    elif np.all(examples[:, -1] == examples[0, -1]):
        return Node(label=examples[0, -1])
    
    elif len(attributes) == 0:
        return Node(label=PLURALITY_VALUE(examples))
    
    else:
        # A = select_best_attribute_GiniIndex(attributes, examples)
        A = select_best_attribute_entropy(attributes, examples)
        print(A) 
        A_index = find_attribute_index(A, examples)
        tree = Node(attribute=A)
        
        for value in np.unique(examples[:, A_index]):
            exs = examples[examples[:, A_index] == value]
            subtree = LEARN_DECISION_TREE(exs, attributes - {A}, examples)
            tree.add_child(value, subtree)
        
        return tree

In [651]:
def print_tree(node, indent=''):
    if node.label is not None:
        print(indent + "Label:", node.label)
    else:
        print(indent + "Attribute:", node.attribute)
        for value, child_node in node.children.items():
            print(indent + "Value:", value)
            print_tree(child_node, indent + "  ")

In [652]:
train_data = np.array(train.head(10))
attributes = {'Alt', 'Bar', 'Fri', 'Hun', 'Pat', 'Price', 'Rain', 'Res', 'Type', 'Est'}
DT = LEARN_DECISION_TREE(train_data, attributes, np.array([]))

print("Decision Tree:")
print_tree(DT)

Pat


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()